In [3]:
import os
print(os.getcwd())
if os.getcwd().endswith("mri_data"):
    os.chdir("..")
    os.chdir("..")
print(os.getcwd())
from src.modelling import DeepModel_Trainer
from src.augmentation import Transformer,None_Transform
from src.data_modules import DataModule,ImagesDataset
from torch import nn
import torch
from torchvision import transforms,models
import nibabel as nib
import numpy as np
from src.evaluation import Evaluation

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ksathmed2/code/Users/pro5d-classification-prolactinoma/modelling/mri_data
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ksathmed2/code/Users/pro5d-classification-prolactinoma


In [4]:
%env "WANDB_NOTEBOOK_NAME" "baseline"
%env WANDB_SILENT=True

base_transformer = Transformer(
    None_Transform(), "select_roi"
)
# %%
def base_mlp():
    model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(
        75264, 1
    )) 
    return model

env: "WANDB_NOTEBOOK_NAME"="baseline"
env: WANDB_SILENT=True


In [ ]:
baseline = DeepModel_Trainer(DataModule(base_transformer), base_mlp)
baseline.train_model("Baseline MLP", "base_mlp", num_epochs=5, test_model=False,batchsize_train_data=16,use_mri_images=True,use_tabular_data=False,num_workers=6,
                     save_model=False,
                     cross_validation=True,lr=0.001,evaluate_test_set=False)

In [ ]:
Evaluation().visualize_slices([4,5],"test")

## Testing

In [5]:
baseline = DeepModel_Trainer(DataModule(base_transformer), base_mlp)
baseline.train_model("Baseline_MLP_Test", "base_mlp", num_epochs=5, test_model=False,batchsize_train_data=16,use_mri_images=True,use_tabular_data=False,num_workers=6,
                     save_model=False,
                     cross_validation=True,lr=0.001,evaluate_test_set=True,project_name="MRI-Modell")

Fold-Iteration:   0%|          | 0/5 [00:00<?, ?fold/s]

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ksathmed2/code/Users/pro5d-classification-prolactinoma/src/modelling.py:203: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_labels = torch.tensor(batch["label"],dtype=torch.float).to(device)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ksathmed2/code/Users/pro5d-classification-prolactinoma/src/modelling.py:319: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_labels = torch.tensor(batch["label"],dtype=torch.float).to(device)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/ksathmed2/code/Users/pro5d-classification-prolactinoma/src/modelling.py:203: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.c